# Logistic regression exercise with Titanic data

## Introduction

- Data from Kaggle's Titanic competition: [data](https://github.com/justmarkham/DAT8/blob/master/data/titanic.csv), [data dictionary](https://www.kaggle.com/c/titanic/data)
- **Goal**: Predict survival based on passenger characteristics
- `titanic.csv` is already in our repo, so there is no need to download the data from the Kaggle website

## Step 1: Read the data into Pandas

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/titanic.csv'
titanic = pd.read_csv(url, index_col='PassengerId')
titanic.head()

## Step 2: Create X and y

Define **Pclass** and **Parch** as the features, and **Survived** as the response.

In [ ]:
feature_cols = ['Pclass', 'Parch']
X = titanic[feature_cols]
y = titanic.Survived

## Step 3: Split the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


## Step 4: Fit a logistic regression model and examine the coefficients

Confirm that the coefficients make intuitive sense.

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
pd.Series(list(zip(feature_cols, logreg.coef_[0])))

## Step 5: Make predictions on the testing set and calculate the accuracy

In [ ]:
# class predictions (not predicted probabilities)
y_pred_class = logreg.predict(X_test)

In [ ]:
# calculate classification accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

### Step 5.1: Let's do some Cross Validation!

In [ ]:
# calculate cross-validated AUC
from sklearn.cross_validation import cross_val_score
print(cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())
print(cross_val_score(logreg, X, y, cv=10, scoring='accuracy').var())

## Step 6: Compare your testing accuracy to the null accuracy

In [ ]:
# this works regardless of the number of classes
#y_test.value_counts().head(1) / len(y_test)
y_test[y_test.values==0].shape[0] / len(y_test)

In [ ]:
# this only works for binary classification problems coded as 0/1
max(y_test.mean(), 1 - y_test.mean())

# Confusion matrix of Titanic predictions

In [ ]:
# print confusion matrix
# Python flips the ordering so that the X-Axis shows "predicted" and the y-axis is the "truth"
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1][1]
TN = confusion[0][0]
FP = confusion[0][1]
FN = confusion[1][0]

In [ ]:
print('True Positives:', TP)
print('True Negatives:', TN)
print('False Positives:', FP)
print('False Negatives:', FN)

In [ ]:
print('False positive rate is ', FP / (float(TN + FP)))
print('True negative rate is ', TN / (float(TN + FP)))
print('True positive rate is ', TP / (float(TP + FN)))
print('False negative rate is ', FN / (float(TP + FN)))

In [ ]:
# calculate the sensitivity AKA the True Positive Rate 
print(TP / float(TP + FN))
print(44 / float(44 + 51))

In [ ]:
# calculate the specificity AKA the True Negative Rate
print(TN / float(TN + FP))
print(105 / float(105 + 23))

In [ ]:
# store the predicted probabilities
y_pred_prob = logreg.predict_proba(X_test)[:, 1]

In [ ]:
# histogram of predicted probabilities
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(y_pred_prob)
plt.xlim(0, 1)
plt.xlabel('Predicted probability of survival')
plt.ylabel('Frequency')

In [ ]:
# increase sensitivity by lowering the threshold for predicting survival to 0.3
# AKA if the probability > 0.3, then predict survival. Otherwise, predict death
import numpy as np
y_pred_class_new = np.where(y_pred_prob > 0.3, 1, 0)
print(np.unique(y_pred_class_new, return_counts=True))
print(np.unique(y_pred_class, return_counts=True))
print(np.unique(y_test, return_counts=True))


In [ ]:
# old confusion matrix
print(confusion)

In [ ]:
# new confusion matrix
print(metrics.confusion_matrix(y_test, y_pred_class_new))

In [ ]:
# new sensitivity (higher than before)
print(63 / float(63 + 32))

In [ ]:
# new specificity (lower than before)
print(72 / float(72 + 56))

In [ ]:
metrics.roc_curve(y_test, y_pred_class)

In [ ]:
# plot ROC curve
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')

**Note**: The number of thresholds = the number of unique probabilities in our prediction array

In [ ]:
# calculate AUC
print(metrics.roc_auc_score(y_test, y_pred_prob))


Besides allowing you to calculate AUC, seeing the ROC curve can help you to choose a threshold that balances sensitivity and specificity in a way that makes sense for the particular context.